# Main Notebook for running code

## Global Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, f1_score
from Functions.implementations import *

## Load Data

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
train_loader,valid_loader, test_loader = get_data_loaders()

Train and evaluate a simple model to test if everything works.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model, train_loader, test_loader, optimizer, criterion, device, num_epochs=5)

## SGD :

In [2]:
import os

#only for google Colab :

# Set a path in your Google Drive
#csv_path = "/content/drive/MyDrive/OptiML/sgd_gridsearch_results.csv"

# For local machine, set the path to your desired location
save_path = os.getcwd() + "/Results/SGD"
csv_path = save_path + "/sgd_gridsearch_results.csv"


epochs = 100
eval_interval = 10

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "momentum"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)
    
set_seed(42) # Set a random seed for reproducibility

In [3]:
from itertools import product


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader,valid_loader, test_loader = get_data_loaders(batch_size=128)

results_grid = {}

# learning_rates = [5e-2, 1e-2, 5e-3, 1e-3]
# momentums = [0.0, 0.6, 0.9, 0.99]

learning_rates = [1e-2, 5e-3]
momentums = [0.9]

# Iterate over all combinations of lr and momentum
for lr, momentum in product(learning_rates, momentums):
    scores, model = train_and_return_evaluation_SGD(
        SimpleCNN,
        lr=lr,
        momentum=momentum,
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval
    )
    results_grid[(lr, momentum)] = scores

    # Extract F1 scores only, for CSV export
    f1_scores = [f1 for (_, _, _, f1) in scores]

    row = [lr, momentum] + f1_scores

    # Append to CSV
    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)
    
    # Save the model
    torch.save(model.state_dict(), save_path + f"/model_lr_{lr}_momentum_{momentum}.pth")


🔧 Training with SGD: lr=0.01, momentum=0.9


KeyboardInterrupt: 

# Adam :

In [2]:
import os

#only for google Colab :

# Set a path in your Google Drive
#csv_path = "/content/drive/MyDrive/OptiML/sgd_gridsearch_results.csv"

# For local machine, set the path to your desired location
save_path = os.getcwd() + "/Results/Adam"
csv_path = save_path + "/adam_gridsearch_results.csv"

# Set number of epochs and evaluation interval
epochs = 100
eval_interval = 10

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "momentum"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

set_seed(42) # Set a random seed for reproducibility

In [12]:
from itertools import product


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, valid_loader, test_loader = get_data_loaders(batch_size=128)

results_grid = {}

# learning_rates = [5e-2, 1e-2, 5e-3, 1e-3]
# betas = [(0.9, 0.999), (0.8, 0.999), (0.9, 0.99)]

learning_rates = [5e-3]
betas = [(0.8, 0.99)]

# Iterate over all combinations of lr and momentum
for lr, betas in product(learning_rates, betas):
    scores, model = train_and_return_evaluation_Adam(
        SimpleCNN,
        lr=lr,
        betas=betas,
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval
    )
    results_grid[(lr, betas[0], betas[1])] = scores

    # Extract F1 scores only, for CSV export
    f1_scores = [f1 for (_, _, _, f1) in scores]

    row = [lr, betas[0], betas[1]] + f1_scores

    # Append to CSV
    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)
    
    # Save the model
    torch.save(model.state_dict(), save_path + f"/model_lr_{lr}_beta1_{betas[0]}_beta2_{betas[1]}.pth")


🔧 Training with Adam: lr=0.005, betas=(0.8, 0.99)
Epoch 10 | Acc=0.7752 | Recall=0.7752 | F1=0.7774
Epoch 20 | Acc=0.9151 | Recall=0.9150 | F1=0.9145
Epoch 30 | Acc=0.9540 | Recall=0.9540 | F1=0.9541
Epoch 40 | Acc=0.9472 | Recall=0.9472 | F1=0.9471
Epoch 50 | Acc=0.9643 | Recall=0.9643 | F1=0.9643
Epoch 60 | Acc=0.9736 | Recall=0.9736 | F1=0.9736
Epoch 70 | Acc=0.9700 | Recall=0.9701 | F1=0.9700
Epoch 80 | Acc=0.9646 | Recall=0.9646 | F1=0.9645
Epoch 90 | Acc=0.9782 | Recall=0.9782 | F1=0.9782
Epoch 100 | Acc=0.9783 | Recall=0.9783 | F1=0.9783

Test Set Evaluation: Acc=0.6104 | Recall=0.6104 | F1=0.6105


Test on colab :

lr = 0.05
- (0.9, 0.999) : F1 ~ 0.18
- (0.8, 0.999) : F1 ~ 0.18
- (0.9, 0.99) : F1 ~ 

lr = 0.01
- (0.9, 0.999) : F1 ~ 
- (0.8, 0.999) : F1 ~ 
- (0.9, 0.99) : F1 ~ 

lr = 0.005
- (0.9, 0.999) : F1 ~ 
- (0.8, 0.999) : F1 ~ 
- (0.9, 0.99) : F1 ~ 

lr = 0.001
- (0.9, 0.999) : F1 ~ 
- (0.8, 0.999) : F1 ~ 
- (0.9, 0.99) : F1 ~ 